## Chapter 1.4: "Inner workings of the dataset"

The first ingredient in our recipe was our data, which lives in our dataset. In the following chapter, we will gain an understanding of ...
1. ... what a dataset is and needs to be capable of,
2. ... how we can build one ourselves,
3. ... and what things we can potentially do with it.

Let's start with 1) - What is a dataset? The dataset is a class. There is two types of dataset, only one of which will be relevant during this course, the so-called map-style dataset. Below, you can find an example of such a dataset, which we build from a couple of example images.

The map-style dataset has to implement three functions. Typically, you will implement more functions, and there is no limit to the amount of functionality you can try and cram into it, but these three are the absolute minimum that must exist.

Interestingly, PyTorch has a basic dataset class, which we can inherit from. However, this is not strictly a requirement, and we can work with the dataset as long as it has the three functions mentioned above.

In [ ]:
import torch, torch.nn as nn
import torchvision, torchvision.transforms.functional as ttf
import numpy as np
import PIL
import matplotlib.pyplot as plt
import os

# In addition to the regular imports, we have added some helper functions.
# They hide some ugly data wrangling, so that our example code looks nicer.
# For testing, you are welcome to use these functions.
# In the end, however, you will have to write these functions yourself.
import sys
sys.path.append("/Volumes/PortableSSD/MLCourse/Course_Materials")
import utility.utils as uu

In [ ]:
class Example_Dataset(torch.utils.data.Dataset):

    """
    The first function any dataset needs is the __init__ function. It is called
    when the dataset instance is created. We can define it to accept any input
    we like, or even none at all. Of course, it does need to take the self
    attribute as its first argument.

    Typically, this function contains things like the dataset path, or maybe
    some settings, if your dataset is sophisticated enough.

    To show you how it works, we have constructed this very bare-bones example.
    Later, you will try your hand at making a more sophisticated one yourselves,
    and, should that not work out, there is also a more sophisticated version
    available in the utility functions we will provide you with.
    """
    def __init__(self, data_dir: str):
        # If we do inherit from PyTorch's Dataset class, let's also call it's
        # __init__ function. This is generally a sensible idea when inheriting.
        super(Example_Dataset, self).__init__()
        
        # Any parameters that the __init__ function later expects, we probably
        # want to remember for later use, so we tack them onto self like so:
        self.data_dir = data_dir

        # If we want to compute something, it might make sense to do it here
        # once, at the start, and remember that result for later.
        # For example, let's check how many training images we have:
        self.files = [os.path.join(self.data_dir, file) for file in os.listdir(self.data_dir) if file.endswith(".png")]
        self.dataset_length = len(self.files)

        return None

    """
    The second function our dataset class requires is the __len__ function.
    If we later want to use len(my_dataset) for example, and of course simply
    for the dataloader to function, we need to figure out how much data is in
    our dataset.

    We could hardcode this value, if we know the amount of images we have.
    Alternatively, we could check the amount of files ending on '.png' in the
    folder containing our data. All that matters, is that the function returns
    an integer (an integer that makes sense, preferably).
    """
    def __len__(self):
        # Since we already computed the amount of data we have above, we can
        # just return this value.

        return self.dataset_length

    """
    The third function is the most important of the functions we need. It is
    called the __getitem__ function. Since the dataloader will pick random
    numbers between 0 and the length of our dataset when preparing a batch,
    this function will take as input an integer. We can name that integer
    whatever we like, although the name idx or index is probably the most
    sensible.

    Funnily enough, despite being the most important function, its output is
    largely up to you. You could return just your image tensor, your image
    tensor and your target, or both and a bunch of other things.
    """
    def __getitem__(self, idx: int):

        # We look up the location of our file first.
        file = self.files[idx]

        # Then, we load up the image, and the target class.
        # Our helper function hides this process.
        # The target classes are hard-coded to represent:
        # 0 => No liver or liver tumors visible.
        # 1 => Liver visible, but no tumors.
        # 2 => Liver and liver tumor visible.
        image, target = uu.lits_classification_loader(file = file)

        # Customarily, images are also resized and normalized
        # so that they have the same shape and all values range
        # from 0 to 1.
        # The former is not necessary for LiTS data, because all
        # images are already 256x256. The latter, we do here.
        image = image - torch.min(image)
        image = image / torch.max(image)

        return image, target


In [ ]:
# Let's quickly confirm that it works.

lits = Example_Dataset(data_dir = "../data/Clean_LiTS/train/volumes/")

example_image, example_target = lits.__getitem__(idx = 420)

print(example_image.size(), example_target)

And that's already it! Fairly straightforward, right?

In order to sharpen your data science skills, it is now your turn to make a dataset class.
The goal of this task is to build the dataset above, but without using our helper functions.
You can look at them if you like, but no blind copying!
- Your dataset class has to implement at least the three functions we mentioned, or more if you want.
- The \_\_getitem\_\_ method you write should return the image (as a tensor) and the target class (also as a tensor).
- The data you want to read in lives at "/data/Clean_LiTS/train/volumes/".
- There is a .csv file at "/data/Clean_LiTS/train/classes.csv" - this file contains your targets.

Here is a few pointers:
- There is three separate folders inside this folder: 'train/', 'val/', and 'test/'. Each of these folders contain a 'volumes' folder. **This is where the CT scans we want to train on are**.
- (There is also another folder in there. It contains segmentations (images of where, if anywhere, liver or tumors are to be seen in an image). You can ignore those folders.)
- Alongside the folders, there is also a classes.csv files for training, validation and testing, respecitvely. Try taking a look at the file. Inside the file, each line has the filename of an image, and information on what is visible in the image. **This is where our targets come from.**

If you want to challenge yourself, try this:
- Loading the data from disk every time \_\_getitem\_\_ is called is inefficient. Try to design a method which loads the dataset once, and keep the loaded images around as an attribute of the class (for example in a dictionary). You could do the same for the targets, too.

In [ ]:
# My very own dataset class







## Chapter 1.5: "Data augmentation, why and how"

**What is data augmentation?**
Have you ever lost your glasses and then squinted, or tried to look through a rainy window? Or looked at a false color image, maybe a forest where the trees are blue and the sky green? You can usually make an educated guess what you are looking at, even though the image you see is different than usual. This is, essentialy, what data augmentation is.

**What is that useful for?**
Let me begin with an anecdote. Say you have pictures of cats and dogs, and want your computer to tell the two apart. How many people you know go to the park with their dogs? I imagine many. Hence, many images of dogs are dogs lying on the grass. The same is generally untrue for cats, at least I have never heard of anyone walking their cat to the park. At any rate, here is what happens when I train a neural network on these images: The computer takes a shortcut. It sees a lot of green and the correct answer for these pictures is always "Dog". It learns "Green = Dog". Ok, fair enough. Now I get some new data. A bunch of people have taken pictures of their cats, sunbathing on the terrace. The garden is in the background. Lots of green. The computer, in its infinite wisdom, will at first guess that these images are of dogs.

This is what we call overfitting. We have overfitted the existence of green in the background as a quintessential part of what makes a dog.

So how can we prevent the computer from taking shortcuts? We try to make shortcuts less useful. If I gave you an image of a dog, but the grass was brown, and the dog green, you could still identify it as a dog, instead of a cat, right? And so should the computer, if we can manage it. So let's also make it train using pictures of cats and dogs where the colours are different or removed. Suddenly, the shortcut solution is no longer useful. This is a practical and useful data augmentation that is used in state-of-the-art image recognition.

In addition to color changes, there is a myriad of other techniques, such as cropping, image rotation or flipping, edge filters, solarization, random noise, and many, many more.

**How do we use data augmentations in practice?**
There is two ways of adding data augmentation during training. Either, you can implement it inside of your dataset, so that it only returns augmented image tensors, or right before feeding your image tensors into your model. Both options are acceptable and come with advantages and disadvantages. You can do either.

If you are particularly eager, or want to try your hand at making image augmentation functions yourself, it can be fun and is definitely good practice. However, PyTorch comes with a large selection of image augmentations right out of the box, and in the following chapter, we will look at how to make use of them. 

In [ ]:
# Let's first get a hold of an image tensor.
tensor_image, tensor_target = lits.__getitem__(idx = 420)
# Make it into an array with the correct dimension order to plot it.
array_image = uu.convert_tensor_to_opencv_array(example_image)
# Plot our base image.
plt.figure()
plt.imshow(array_image, cmap = "bone")
plt.xlim((0, 256))
plt.ylim((0, 256))

In [ ]:
# Now let's make some data augmentations happen!
# PyTorch offers us two ways of doing so: We can use the functional
# transforms, or the class-based transforms.

# Functional transforms are very straightforward and look like this:
transformed_tensor = ttf.rotate(tensor_image, angle=90)

# Let's check out what we've done!
transformed_array = uu.convert_tensor_to_opencv_array(transformed_tensor)
plt.figure()
plt.imshow(transformed_array, cmap = "bone")
plt.xlim((0, 256))
plt.ylim((0, 256))

In [ ]:
# Alternatively, there is class-based transforms, which look like this:
rotation = torchvision.transforms.RandomRotation(90)

# We have just made an instance of a class. The class is "callable", which
# essentially means that we can call it like a function (because it has a
# __call__ method).
transformed_tensor = rotation(tensor_image)

# Let's check out what we've done!
transformed_array = uu.convert_tensor_to_opencv_array(transformed_tensor)
plt.figure()
plt.imshow(transformed_array, cmap = "bone")
plt.xlim((0, 256))
plt.ylim((0, 256))

# Importantly, the function and class versions occasionally work differently.
# Always pay attention to what the function or class says it does!

In [ ]:
# Both ways of writing augmentations have good reasons to exist. Generally,
# however, the class variant is preferred. It also comes with a neat way of
# combining a number of data augmentations into a single callable.

# We can make a list of augmentations we want. This list here contains a
# blur, and a randomized vertical flip. To showcase the flip, we have set
# the probability of the flip being applied to 1.
augmentation_list = [
    torchvision.transforms.GaussianBlur(kernel_size = [3, 3], sigma = 0.1),
    torchvision.transforms.RandomVerticalFlip(p = 1.0)
    ]

# This is how we get our single callable.
my_transforms = torchvision.transforms.Compose(transforms = augmentation_list)

# We can apply it like before ...
transformed_tensor = my_transforms(tensor_image)

# ... and we can see what it has done.
transformed_array = uu.convert_tensor_to_opencv_array(transformed_tensor)
plt.figure()
plt.imshow(transformed_array, cmap = "bone")
plt.xlim((0, 256))
plt.ylim((0, 256))

In [ ]:
# Try to play around with the augmentations a little and see what they do.
# You are welcome to check out a color image, too, because those offer even
# more alternatives.







For those who want to dive a little deeper:
The 'forward pass' is what is implicitly called whenever we ask our model to do something. It gives us our output and accumulates the gradients which we can later use to optimize our parameters (which happens during the backward pass).

Particularly cool is that the class variant actually implements a forward pass for the entire augmentation process. This means that, depending on what kind of augmentations or other things we throw at our image tensor, that even the augmentation process can be differentiable! Now, most of the time this is absolutely useless. But if you decide to stay in data science and work with PyTorch a lot, and build your own clever little augmentation policies and so on, you will come to appreciate this fact.

## Chapter 1.6: "The model"

Now comes the créme de la créme of training a neural network - the neural network.

All we have seen of the neural network so far, is that we stuff a tensor with shape N x C x H x W into the front end, and get a tensor N x C back out. During the course, you have probably been shown what a typical network consists of (things like Convolutional Layers, Fully Connected Layers, Non-Linear Activation Functions, Batch Normalization, etc.), and what these things do, mathematically speaking.

But how do programmers build them? As almost always, the answer is *classes*. Any neural network in PyTorch is a class, which does two things:
- It inherits from a PyTorch class named *torch.nn.Module*.
- It implements an \_\_init\_\_ and a forward method.

The inheritance guarantees that any neural network (and often even its components), follow a common structure. Methods like *model.parameters()* that we have previously used would not function without this inheritance.
The forward function is implicitly called whenever we stuff something into our model and tell it to make a prediction for us. The corresponding backward path, which computes the gradients for each parameter, is mercifully something that PyTorch does *automatically*.

In [ ]:
# It's time we built a model ourselves.

# We start with a class that inherits from torch.nn.Module
class Example_Model(torch.nn.Module):

    # We want an init method, and at the very least we want
    # to call the init function of our parent class aswell.
    def __init__(self):
        super(Example_Model, self).__init__()

    # We also want a forward method, so our model actually
    # does something when we call it.
    def forward(self, x):
        return x

# We make an instance of our model, ...
my_first_model = Example_Model()

# ... quickly grab an example "image" to feed in, ...
some_zeroes = torch.rand((16, 1, 256, 256))

# ... and tell our model to make a prediction.
prediction = my_first_model(some_zeroes)

print(prediction.size())

In [ ]:
# And that's already it! 

# ... or is it?
# So far, our model has not done anything, of course, it just spat
# our input tensor back out. Let's give it some learnable parameters.

class Example_Model(torch.nn.Module):

    def __init__(self):
        super(Example_Model, self).__init__()

        # This time, we specify some components.
        # You could write components yourself, but in practice,
        # PyTorch has pretty much all you will ever need.

        # Linear is the name of the fully connected layer in PyTorch.
        # The number of in-going signals will be as many as we have
        # pixels in our toy model.
        # The number of out-going features is up to our choosing.
        self.fc1 = torch.nn.Linear(in_features = 16*1*(256**2), out_features = 3)

        # We will learn more about these components in a little bit.

    def forward(self, x):
        x = x.flatten()
        # This time, we actually use our predefined layer in our
        # forward pass.
        x = self.fc1(x)
        return x

my_first_model = Example_Model()
some_zeroes = torch.rand((16, 1, 256, 256))
prediction = my_first_model(some_zeroes)
print(prediction.size())

In [ ]:
#You can also look at what building blocks your model is composed of:
print(my_first_model)

For those who want to dive deeper: 

When we wrote 'self.fc1 = nn.Linear(...)', we added a module to our model (duh), and the class can henceforth refer to this module by its name (e.g. 'x = self.fc1(x)' works). The model also knows to collect gradients for and to update the parameters of these modules. If you want to build sophisticated models, sometimes you will not handcraft everything, but rather make a couple of building blocks, and want to add a variable number of them to your model. For such cases, there is a function named 'self.add_module(name, child_module)'. In essence, the following two things are equivalent:
- self.add_module("fc1", nn.Linear(...))
- self.fc1 = nn.Linear(...)  

However, sometimes it is very convenient being able to fill in the left side of the equation dynamically.

## Chapter 1.7 - "Model lego"

During the precursor seminar, you have probably learnt about some of the concepts behind neural networks. If not, or if you do not remember, here is a little crash course on what building blocks we have, how they work, and how we can easily conceptualize what they do.

There is far more things that PyTorch can do, and it would be quite impossible to showcase all of them here. Others, like BatchNorm, we will learn about as we implement architectures from the more recent neural network milestone papers. If you come across something you haven't heard before in a paper, or have an idea for something you'd like to try, there is a good chance that you can find this operation already fully implemented in PyTorch - try googling it! You will find that the PyTorch documentation is quite good, and that you can discover some very useful features this way.

**Linear layers** - The simplest and oldest component of a neural network is the Linear layer, often called a Fully Connected layer. A fully connected layer is defined by a number of in-going and out-going connections. The number of in-going connections, naturally, is the same as the number of out-going connections from the layer before. The number of out-going connections can be is chosen so that every node in our layer is connected to every node in the following layer.

The Linear layer is called linear because each out-going signal equals the sum of all products of a node's weight and in-going signal, plus a bias term per node. This makes it, mathematically speaking, a linear transformation plus a translation (also called an affine transformation):
$y = x*A^T +b$

In [ ]:
# In PyTorch, a Linear layer that takes 20 input signals and gives 10 output signals is made like this:

fc1 = torch.nn.Linear(in_features = 20, out_features = 10)

**Activation functions** - Where neural networks are concerned, Linear layers are nice, but come with an inherent problem: Any combination of two affine transformations, is itself an affine transformation. Consequently, neural networks do not really gain much from multiple Linear layers that one Linear layer couldn't have done, and also generally have trouble capturing higher-order polynomial dependencies in data. However, there is a solution to this problem! A so-called activation function is commonly added after every Linear layer (and typically also after convolutions, which we will get to later), and its purpose is to introduce some form of non-linear transformation between our affine transformations. Now, suddenly, having multiple layers is quite useful, and the issues capturing non-linear and particularly higher-order polynomial dependencies are lessened.

There is a multitude of different activation functions, and historically, different ones have been used for different reasons. The most common among these activation functions are:
- Tangens Hyperbolicus: $tanh(x) = \frac{e^{2x}-1}{e^{2x}+1}$
- Sigmoid: $\sigma(x) = \frac{1}{(1+e^{-x})}$
- ReLU: $R(x) = \{0 $ if $ x<0 $, else $ x\}$
- Swish: $S(x) = \frac{x}{(1+e^{-ßx})}$

Commonly, ReLU is used for most applications, and we recommend you stick to ReLU as well.

In [ ]:
# In PyTorch, these functions can be invoked via:

# This is a class instance of the ReLU class
relu = torch.nn.ReLU()

# It does almost nothing except apply the ReLU function to
# your tensor, and being differentiable.
some_tensor = torch.randn((16,3,256,256))
some_tensor = relu(some_tensor)

**Convolution layers** - Conceptually, you can imagine a convolution operation as asking the question 'Where on my image do I have the feature **f**'? The (typical 2D) convolution operator (or kernel) is a tensor of shape M x N. The convolution operation assigns the value of a pixel in the result image by computing the element-wise product of our input image in an area around a so-called *anchor pixel* and the convolution kernel, and then summing. This process is repeated for all pixels in the original image:  
  
$I_{new}[x, y] = (I * K)[x, y] = \sum_{a}\sum_{b} K[a, b]*I[x - a, y - b]$  
  
with $I$ being our original image, $I_{new}$ being the result image, and $K$ being the convolution kernel.
*In layman's terms, if you draw a little circle into the kernel, then the resulting image is essentially a map of where in the original image you can find circles such as the one in the kernel*. It is therefore often call a *feature map* or an *activation map*.

In a modern neural network, convolutional layers are ubiquitous. The computer will, over the course of the training, adapt the contents of the convolution kernels in order to best represent and categorize the images it sees. Features in early convolutional layers correspond to the actual content of the image and make intuitive sense to a human observer - to classify cars, you can expect to be needing a couple of differently angled straight lines, some smooth curves, and circles for the wheels, for example. The features in later layers are less (or not at all) interpretable to human eyes. At that point, the image that the computer has, typically consists of activation maps of activation maps of ... you get the gist. 

It is still possible to visualize what these represent, and some of the features are surprisingly human-interpretable (such as an entire car wheel, or the texture of a brick wall), while others look like LSD-induced hallucinations. If we have some time at the end of the course, we may try our hands at this visualization ourselves, and take a look at how your neural networks see the world.

Convolution operations are defined by their number of in-going and out-going signal channels. At the beginning of a neural network, the in-going channel number is typically 3 (for RGB images) or 1 (for grayscale images). The out-going channel number can be freely chosen. Convolution operations can have any kernel size, but typically its height and width are chosen as odd numbers. This is done because the convolution kernel is applied to the area around the anchor pixel in our original image - a kernel with odd dimensions can be centered exactly on this anchor, while one with even dimensions can not. The latter case causes interesting systematic errors, which are fun to look at, but have a tendency to ruin the performance of the neural network.

In [ ]:
# In PyTorch, these functions can be invoked via:

# This is a class instance of the ReLU class
conv = torch.nn.Conv2d(in_channels = 3, out_channels = 8, kernel_size = (3, 3))

# It does almost nothing except apply the ReLU function to
# your tensor, and being differentiable.
some_tensor = torch.randn((16,3,256,256))
some_tensor = conv(some_tensor)

**Pooling operations** - Conceptually, a pooling operation is quite simple. The kernel of the pooling operator has a certain shape, M x N. We compute a new image from a pooling operation by starting at a pixel, and applying the pooling operator to the pixel and its surroundings. The result is the value of the a new pixel. This process is repeated for every pixel in the image and you get a new image. Typically, the pooling operator either performs the max() or the mean() operation on the area it is applied to. These pooling ops are name MaxPool and AvgPool, respectively.

Very often, something called a 'stride' is applied during the pooling operation. 'A stride of *s*' means that in order to get a new image, we do not iterate over every pixel and perform pooling on its respective surrounding, but that we instead skip *s* rows and columns on each step. Typically, stride and pooling operator shapes are chosen to be the same, so that no pixel is ignored during the calculations, and so that no pixel contributes to the new image more than once. Consequently, the new image after a pooling operation will be downscaled by a factor *s*. Since, in a sense, the new image still contains *all, or most of the relevant* information from the original image, strided pooling is often used for exactly this downscaling effect. This reduces the amount of computation needed for later steps, and helps extracting the most relevant information from an image.

If you try to imagine applying a 2x2 pooling operator to, say, a 5x5 image, do you notice a problem? No matter how you define the 2x2 surrounding of your current pixel, some pixels you need to calculate the result of the pooling operator will not exist - they would be outside the bounds of our image! This is a very common occurence, and pooling operations perform something called *padding*. Padding extends the image by the necessary pixels, just for the calculations of pixels that actually exist. Typically, the image is extended using pixels which exclusively have the values zero.

In [ ]:
# In PyTorch, these pooling operations can be invoked via:

# This is a class instance of the MaxPool2d class
max_pool = torch.nn.MaxPool2d(kernel_size = (2, 2), stride = 2)

# It does almost nothing except apply the ReLU function to
# your tensor, and being differentiable.
some_tensor = torch.randn((16,3,256,256))
some_tensor = max_pool(some_tensor)

In [ ]:
# And now it's your turn. Try to make a model that performs some
# sensible mathematical operations that you've learned about in
# the course. For example, try adding in a convolutional layer
# or two, a fully connected layer or two, and nonlinearities
# such as ReLUs or tanhs.

# This dummy batch (batch size 16, one channel, 256x256 pixels)
# will be your input. Try to see if the output of your model is
# what you expected it to be.
dummy_batch = torch.rand((16, 1, 256, 256))

# Remember that you can use some_tensor.size() to check the
# current dimensions of your tensor, so that you keep track of
# what the things you've just added do.

# When you're done, try checking if your model is capable of
# the same methods you are used to, such as model.parameters(),
# and whether the layers you created show up.









## Chapter 1.8: "Putting it all together"

We have all the pieces. We have a recipe, with which we can train a model using some data. We have made ourselves a dataset, using the LiTS data. And we know how to make our own, small, models.

Try to stitch together all the individual parts that you have built. You are allowed to just copy your previous solutions. Try using as many of the things you have learnt about as possible, and see if your neural network is capable of training to recognize liver cancer.

In [ ]:
# My first end-to-end solution





